# Photo Spec Calculator
This tool is used to calculate unknown photo specs (pixel size and DPI or photo size) from known specs (DPI or photo size).

In [4]:
import cv2
import os
import statistics

from utils import sprint

In [5]:
def calculate_dpi(photo_size, img_size):
    """
    Calculate dots per inch (DPI) given a physical photo size in millimeters and a digital image size
    in pixels.
    @param {tuple} photo_size Physical photo size (length, width) in millimeters.
    @param {tuple} img_size Digital image size (length, width) in pixels
    @return {float} Calculated DPI of the image.
    """
    # Convert photo size from millimeters to inches
    l_in = photo_size[0] / 25.4
    w_in = photo_size[1] / 25.4

    l_dpi = img_size[0] / l_in
    w_dpi = img_size[1] / w_in

    return statistics.mean((l_dpi, w_dpi))

In [6]:
def calculate_photo_size(dpi, img_size):
    """
    Calculate physical photo size in millimeters given dots per inch (DPI) and digital image size
    in pixels.
    @param {int} dpi Dots per inch that the photo was scanned at.
    @param {tuple} img_size Digital image size (length, width) in pixels
    @return {tuple} Calculated photo size (length, width).
    """
    # Calculate photo size in inches
    l_in = img_size[0] / dpi
    w_in = img_size[1] / dpi
        
    # Convert photo size to millimeters
    l_mm = l_in * 25.4
    w_mm = w_in * 25.4
        
    return (l_mm, w_mm)

In [7]:
def calculate_pixel_size(photo_size, img_size):
    """
    Calculate pixel size in millimiters given physical photo size in millimeters and digital image size
    in pixels.
    @param {tuple} photo_size Physical photo size (length, width) in millimeters.
    @param {tuple} img_size Digital image size (length, width) in pixels.
    @return {tuple} Calculated ixel size (length, width).
    """
    l_pixel = photo_size[0] / img_size[0]
    w_pixel = photo_size[1] / img_size[1]
    
    return (l_pixel, w_pixel)

In [8]:
def calculate_photo_specs(path, dpi=None, photo_size=None, silent=True):
    """
    Pass a photo path and either DPI or photo size to calculate 
    pixel size and the other parameter. 
    @param {str} path The path to the photo image.
    @param {int} dpi The scanning resolution of the image in dots
    per inch. If DPI is not provided, it will be calculated based on
    photo size. DPI is calculated as the mean of the length and width
    DPI. 
    @param {tuple of ints} photo_size The physical size of the scanned 
    image in millimeters format (length, width). If photo size is not
    provided, it will be calculated based on DPI.
    @return {tuple} Photo specs in the format ((pixel length in mm, pixel width in mm), dpi, (length in mm, width in mm))
    """
    if all((dpi, photo_size)):
        raise Exception("Provide only dpi or photo_size argument, not both.")
    
    img = cv2.imread(path)
    l, w, _ = img.shape
    img_size = (l, w)
    
    if dpi:
        calculating = "Photo size"
        photo_size = calculate_photo_size(dpi, img_size)
    
    elif photo_size:
        calculating = "DPI"
        dpi = calculate_dpi(photo_size, img_size)
    
    else:
        raise Exception("Provide either dpi or photo_size argument.")
        
    pixel_size = calculate_pixel_size(photo_size, img_size)
    
    sprint(f"""
        Image resolution (px): {l} (L) x {w} (W)
        Calculated: {calculating}\n
        DPI: {dpi}
        Photo length (mm): {photo_size[1]}
        Photo width (mm): {photo_size[0]}
        Pixel length (mm): {pixel_size[0]}
        Pixel width (mm): {pixel_size[1]}
        """, silent)
    
    return(pixel_size, dpi, photo_size)

## Example

In [12]:
photo_path = os.path.join("data", "DWW_3FF_117.tif")

pixel_size, dpi, _ = calculate_photo_specs(photo_path, photo_size=(224, 224), silent=False)


        Image resolution (px): 1886 (L) x 1885 (W)
        Calculated: DPI

        DPI: 213.80223214285712
        Photo length (mm): 224
        Photo width (mm): 224
        Pixel length (mm): 0.11876988335100742
        Pixel width (mm): 0.11883289124668435
        
